# Rust
En este notebook se va a analizar la viabilidad de utilizar la tecnología **rust** con la que a priori, la cual tiene dentro de sus beneficios los siguientes:

- Misma eficiencia que con `C++`.
- Seguridad en el uso de memoria.
- Integración práctica con Python.

## Primeros pasos con `rust`
Para empezar se empezó [instalando](https://www.rust-lang.org/tools/install) `rust` de una forma muy sencilla a través de `curl` y posteriormente se siguió el [hola mundo](https://www.rust-lang.org/learn/get-started) que la documentación proveé el cual contiene lo siguiente:

```rs
fn main() {
    println!("Hello World!");
}
```

El cual pudo ser ejecutado después haber inicializado el proyecto con `cargo new <nombre_proyecto>` y luego corriéndolo con `cargo run`.

## Integración con Python
**PyO3** son integraciones de `Rust` con `Python` para poder crear, correr e interactuar con código `Python` desde binarios en `Rust` [[1](https://pyo3.rs/v0.22.2/)].

Un ejemplo básico es a través de la librería `maturin` que se puede instalar a través de `pip` y permite construir y publicar paquetes `Python` basados en `Rust` con una mínma configuración a través de un archivo `Cargo.toml`

```toml
[package]
name = "stmeasures"
version = "0.1.0"
edition = "2021"

[lib]
name = "stmeasures"
# "cdylib" is necessary to produce a shared library for Python to import from.
#
# Downstream Rust code (including code in `bin/`, `examples/`, and `tests/`) will not be able
# to `use string_sum;` unless the "rlib" or "lib" crate type is also included, e.g.:
# crate-type = ["cdylib", "rlib"]
crate-type = ["cdylib"]

[dependencies.pyo3]
version = "0.22.2"
features = ["extension-module"]
```

y el archivo `src/lib.rs`:
```rs
use pyo3::prelude::*;

/// Formats the sum of two numbers as string.
#[pyfunction]
fn sum_as_string(a: usize, b: usize) -> PyResult<String> {
    Ok((a + b).to_string())
}

/// A Python module implemented in Rust.
#[pymodule]
fn stmeasures(m: &Bound<'_, PyModule>) -> PyResult<()> {
    m.add_function(wrap_pyfunction!(sum_as_string, m)?)?;

    Ok(())
}
```

Haciéndo algunas modificaciones (si lo son necesarias) pertinentes, después de ejecutar en terminal `maturin develop`, es posible utilizar esta librería recién creada.

In [1]:
import stmeasures

stmeasures.sum_as_string(20, 12)

'32'

Y ahora se agrega la siguiente función a nuestra librería:

```rs
#[pyfunction]
fn distance(p: Vec<f64>, q: Vec<f64>) -> PyResult<f64> {
    if p.len() != q.len() {
        return Err(PyErr::new::<pyo3::exceptions::PyValueError, _>(
            "Error: Arrays must always have the same size."
        ));
    }

    let squares_sum = p.iter()
        .zip(q.iter())
        .map(|(a, b)| (a - b).powi(2))
        .sum::<f64>();

    Ok(squares_sum.sqrt())
}
```

Sin olvidar agregar `m.add_function(wrap_pyfunction!(distance, m)?)?;` a nuestro módulo, y con esto ya es posible utilizar esta función de distancia euclideana.

In [2]:
stmeasures.distance([1, 2], [3, 4])

2.8284271247461903